In [3]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import os
from six.moves import urllib
import zipfile

In [28]:
DOWNLOAD_ROOT = "http://ceprdata.org/wp-content/cps/data/"
CEPR_PATH = "data/teachers"
CEPR_FILE_NAME = "cepr_org_{}.zip"
DOWNLOAD_URL = DOWNLOAD_ROOT + CEPR_FILE_NAME
def fetch_cepr_data(start_year=1979, end_year=2016):
    if not os.path.isdir(CEPR_PATH):
        os.makedirs(CEPR_PATH)
    for i in range(start_year, end_year+1):
        zip_path = os.path.join(CEPR_PATH, CEPR_FILE_NAME.format(i))
        urllib.request.urlretrieve(DOWNLOAD_URL.format(i), zip_path)
        cepr_zip = zipfile.ZipFile(zip_path, 'r')
        cepr_zip.extractall(CEPR_PATH)
        cepr_zip.close()

fetch_cepr_data(start_year=1979, end_year=2016)

In [4]:
def load_cepr_data(start_year=1979, end_year=2016):
    composite_cepr_data = pd.read_stata("data/teachers/cepr_org_{}.dta".format(start_year), 
                                        convert_categoricals=False)
    for i in range(start_year+1, end_year+1):
        year_data = pd.read_stata("data/teachers/cepr_org_{}.dta".format(i), convert_categoricals=False)
        composite_cepr_data = composite_cepr_data.append(year_data)
        
    return composite_cepr_data

cepr_data = load_cepr_data(start_year=1994, end_year=2016)

KeyboardInterrupt: 

In [308]:
pd.io.stata.StataReader('data/teachers/cepr_org_2016.dta').variable_labels()

{'abpaid': 'Paid for time off last week',
 'absent79': 'Why absent last week',
 'absent94': 'Why absent last week',
 'age': 'Age',
 'agric': 'Agriculture',
 'arrived': 'Year entered US',
 'blsimph': 'BLS allocated hourly earnings',
 'blsimpt': 'BLS allocated weekly hours',
 'blsimpw': 'BLS allocated weekly earnings',
 'cbsasz': 'CBSA Size',
 'centcity': 'Central city',
 'ch02': 'Child 0-2',
 'ch05': 'Child 0-5',
 'ch1417': 'Child 14-17',
 'ch35': 'Child 3-5',
 'ch613': 'Child 6-13',
 'citizen': 'US citizen',
 'cmsacode05': 'CSA Code',
 'cmsacode14': 'CSA Code',
 'cow1': 'Class of Worker, 1st job',
 'cow2': 'Class of Worker, 2nd job',
 'docc03': '2-digit occupation',
 'docc70': '2-digit occupation',
 'docc80': '2-digit occupation',
 'earnwke': 'Weekly pay',
 'educ': 'Education level',
 'educ92': 'Education level, post-1991',
 'empl': 'Employed',
 'entrant': 'Entrant',
 'faminc': 'Family income band',
 'famrel': 'Family relation',
 'famrel84': 'Family relation',
 'famrel94': 'Family rela

In [240]:
# Getting responses from people who are employed,
# working at least 35 hours a week,
# and between the ages of 18 and 64
cepr_data_final = cepr_data[(cepr_data['age'] >= 18) & 
                            (cepr_data['age'] <= 64) &
                            (cepr_data['hourslwm'] >= 35) &
                            (cepr_data['selfemp'] == 0.0)]

# making a date time object for each month
cepr_data_final['year_month'] = pd.to_datetime(cepr_data_final['year'].astype(str) + '-'+ cepr_data_final['month'].astype(str))

/Users/Dayo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [236]:
# Loading and transforming cpi data for join
cpi_data = pd.read_csv('data/teachers/cpi_u_rs.csv', index_col='year')

cpi_list = []
date_list = []
for i in range(1994, 2017):
    for j in range(0, 12):
        cpi_list.append(cpi_data.loc[i][j])
        date_list.append(str(i) + '-' + str(j+1))

cpi_dict = {
    'year_month': date_list,
    'cpi': cpi_list
}
cpi_df = pd.DataFrame(cpi_dict)
cpi_df['year_month'] = pd.to_datetime(cpi_df['year_month'])
cpi_df.set_index(['year_month'], inplace=True)

In [245]:
# Joining on cpi data to calculate real income
cepr_data_final = cepr_data_final.join(cpi_df, on='year_month')

cepr_data_final['2016_weekpay'] = (cepr_data_final['weekpay'] * 352.6) / cepr_data_final['cpi']

In [318]:
# Getting public teachers by Census Occupation Code 
public_teachers = ((cepr_data_final['publoc'] == 1.0) &
                   (cepr_data_final['educ'] >= 4.0) &
                    ((cepr_data_final['occ12'] == 2310.0) |
                    (cepr_data_final['occ12'] == 2320.0) | 
                    (cepr_data_final['occ11'] == 2310.0) | 
                    (cepr_data_final['occ11'] == 2320.0) |
                    (cepr_data_final['occ03'] == 2310.0) |
                    (cepr_data_final['occ03'] == 2320.0) |
                    (cepr_data_final['occ80'] == 156.0) | 
                    (cepr_data_final['occ80'] == 157.0))) 
public_teachers_df = cepr_data_final[public_teachers]

college_graduates = cepr_data_final[(cepr_data_final['educ'] >= 4.0) & 
                                   ~public_teachers]

In [359]:
public_teachers_df.pivot_table('2016_weekpay', index='year', columns='female')

female,0,1
year,,
1994,1224.010491,1107.495387
1995,1208.576348,1086.151546
1996,1207.472069,1078.608880
1997,1232.379371,1078.353660
1998,1215.210430,1106.656281
1999,1243.312423,1109.190957
2000,1242.408837,1098.065340
2001,1210.346911,1096.246291
2002,1210.558851,1105.555751


In [360]:
college_graduates.pivot_table('2016_weekpay', index='year', columns='female')

female,0,1
year,,
1994,1386.159464,1032.378581
1995,1391.345637,1036.795769
1996,1378.763585,1034.925599
1997,1389.641268,1051.937201
1998,1496.139613,1097.520940
1999,1537.825865,1119.553792
2000,1551.481944,1144.196864
2001,1574.513572,1158.105423
2002,1572.450262,1160.758477


In [316]:
cepr_data_final.pivot_table('2016_weekpay', index='year')

year
1994     867.194900
1995     871.049036
1996     869.378531
1997     878.944266
1998     915.665895
1999     932.908714
2000     946.790042
2001     962.011653
2002     970.403298
2003     978.769485
2004     987.498478
2005     992.873559
2006     997.836113
2007    1010.579463
2008    1029.778861
2009    1036.428343
2010    1036.549858
2011    1039.172069
2012    1038.950043
2013    1040.491157
2014    1029.830147
2015    1024.800025
2016    1029.236878
Name: 2016_weekpay, dtype: float64

In [348]:
# separating teachers by degree obtained
public_teachers_ba = public_teachers_df[(public_teachers_df['year'] > 2011) 
                                        & (public_teachers_df['educ92'] == 13)]
public_teachers_ma = public_teachers_df[(public_teachers_df['year'] > 2011) 
                                        & (public_teachers_df['educ92'] == 14)]

In [349]:
# separating college graduates by degree obtained
college_graduates_ba = college_graduates[(college_graduates['year'] > 2011) 
                                        & (college_graduates['educ92'] == 13)]
college_graduates_ma = college_graduates[(college_graduates['year'] > 2011) 
                                        & (college_graduates['educ92'] == 14)]

In [1]:
state_values = public_teachers_ma.pivot_table('2016_weekpay', index='state', aggfunc='count')
for value in state_values:
    print(value)

NameError: name 'public_teachers_ma' is not defined

In [22]:
# loading and modifying car loan data
car_loan_df = pd.read_csv('data/TERMCBAUTO48NS.csv')
car_loan_df['VALUE'].fillna(0, inplace=True)
car_loan_df['DATE'] = pd.to_datetime(car_loan_df['DATE'])
car_loan_df['YEAR'] = car_loan_df['DATE'].dt.year
car_loan_df.pivot_table('VALUE', index='YEAR')

YEAR
1972    3.654545
1973    3.403333
1974    3.656667
1975    3.787500
1976    3.690000
1977    3.639167
1978    3.673333
1979    4.005000
1980    4.766667
1981    5.513333
1982    5.608333
1983    4.639167
1984    4.570000
1985    4.303333
1986    3.776667
1987    3.484167
1988    3.618333
1989    4.022500
1990    3.927500
1991    3.712500
1992    3.096667
1993    2.695833
1994    2.705000
1995    3.190000
1996    3.015833
1997    3.005833
1998    2.908333
1999    2.813333
2000    3.113333
2001    2.833333
2002    2.538333
2003    2.311667
2004    2.200833
2005    2.357500
2006    2.571667
2007    2.589167
2008    2.340833
2009    2.239167
2010    2.068333
2011    1.910833
2012    1.637500
2013    1.475000
2014    1.412500
2015    1.397500
2016    1.433333
2017    2.260000
Name: VALUE, dtype: float64